## effect size sharing for multi-gene qtls

say we have a qtl that impacts multiple genes. How does the fractional effect of that qtl on each gene vary? w.r.t for instance distance from qtl to tss? maybe tie in ABC connections here? Maybe build a model that predicts how qtl impact on each gene will be split based on gene baseline expression, distance, ect

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 
import os
import ast
import re

import sys
sys.path.append('/home/klawren/oak/pcqtls/workflow/scripts')
from notebook_helper_functions import *

import upsetplot as up
from tqdm.auto import tqdm  # for notebooks

# Create new `pandas` methods which use `tqdm` progress
# (can use tqdm_gui, optional kwargs, etc.)
tqdm.pandas()

In [2]:
# get outputs from a config file
prefix = '/home/klawren/oak/pcqtls'
import yaml
config_path= f'{prefix}/config/proteincoding_main.yaml'
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

# load in the tissue ids 
tissue_df = load_tissue_df(config)
tissue_ids = list(tissue_df['Tissue'])

# select just 1 tissue id to do for now 
my_tissue_id = 'Cells_Cultured_fibroblasts' # because only one cell type
#tissue_ids = [tissue_id]

### load in data
effect sizes for qtls
overlap dfs
gencode for tss starts

In [3]:
e_nominal = load_e_nominal(config, my_tissue_id, chr_id=22)
pc_nominal = load_pc_nominal(config, my_tissue_id, chr_id=22)
overlap = load_overlap(config, my_tissue_id)
pc_susie = load_pc_susie(config, my_tissue_id)

# set e_nominal to be index based on variant-cluster pairs
e_nominal_cid = e_nominal.set_index('var_cluster')

In [5]:
pc_susie

,phenotype_id,variant_id,pip,af,cs_id,var_cluster,cs_num,pc_num,lead_variant_id
0,ENSG00000187583.10_ENSG00000187961.13_pc1,chr1_966179_G_A_b38,0.999976,0.838509,ENSG00000187583.10_ENSG00000187961.13_pc1_1,chr1_966179_G_A_b38_ENSG00000187583.10_ENSG000...,1,1,chr1_966179_G_A_b38
1,ENSG00000160072.19_ENSG00000197785.13_pc1,chr1_1497758_C_T_b38,0.249760,0.010352,ENSG00000160072.19_ENSG00000197785.13_pc1_1,chr1_1497758_C_T_b38_ENSG00000160072.19_ENSG00...,1,1,chr1_1497758_C_T_b38
2,ENSG00000160072.19_ENSG00000197785.13_pc1,chr1_1499000_C_A_b38,0.249760,0.010352,ENSG00000160072.19_ENSG00000197785.13_pc1_1,chr1_1499000_C_A_b38_ENSG00000160072.19_ENSG00...,1,1,chr1_1497758_C_T_b38
3,ENSG00000160072.19_ENSG00000197785.13_pc1,chr1_1520463_T_C_b38,0.249760,0.010352,ENSG00000160072.19_ENSG00000197785.13_pc1_1,chr1_1520463_T_C_b38_ENSG00000160072.19_ENSG00...,1,1,chr1_1497758_C_T_b38
4,ENSG00000160072.19_ENSG00000197785.13_pc1,chr1_1534481_C_T_b38,0.249760,0.010352,ENSG00000160072.19_ENSG00000197785.13_pc1_1,chr1_1534481_C_T_b38_ENSG00000160072.19_ENSG00...,1,1,chr1_1497758_C_T_b38
...,...,...,...,...,...,...,...,...,...
31900,ENSG00000054148.17_ENSG00000232434.2_pc2,chr9_136872390_A_G_b38,0.327839,0.010352,ENSG00000054148.17_ENSG00000232434.2_pc2_1,chr9_136872390_A_G_b38_ENSG00000054148.17_ENSG...,1,2,chr9_136867040_A_G_b38
31901,ENSG00000054148.17_ENSG00000232434.2_pc2,chr9_136872914_A_C_b38,0.327839,0.010352,ENSG00000054148.17_ENSG00000232434.2_pc2_1,chr9_136872914_A_C_b38_ENSG00000054148.17_ENSG...,1,2,chr9_136867040_A_G_b38
31902,ENSG00000198569.9_ENSG00000233198.3_pc1,chr9_137224523_G_A_b38,0.482402,0.143892,ENSG00000198569.9_ENSG00000233198.3_pc1_1,chr9_137224523_G_A_b38_ENSG00000198569.9_ENSG0...,1,1,chr9_137224523_G_A_b38
31903,ENSG00000198569.9_ENSG00000233198.3_pc1,chr9_137225120_C_T_b38,0.281381,0.145963,ENSG00000198569.9_ENSG00000233198.3_pc1_1,chr9_137225120_C_T_b38_ENSG00000198569.9_ENSG0...,1,1,chr9_137224523_G_A_b38


take the subset of varaints in pcqtl determined credible sets, and calculate the varience-explained on a per gene basis for each gene in the cluster and for the cluster-pc overall

In [4]:
# get a subset of variants that ended up in credible sets
e_nominal_pcsusie_subset = e_nominal[e_nominal['var_cluster'].isin(pc_susie['var_cluster'])]

# I want to get a pip weighted variance for each egene
e_nominal_pcsusie_subset['var_egene_cluster'] = e_nominal_pcsusie_subset['variant_id'] + '_' + e_nominal_pcsusie_subset['phenotype_id']

# merge in the data from the susie finemapping
e_nominal_pcsusie_subset = pd.merge(e_nominal_pcsusie_subset, pc_susie[['cs_id', 'var_cluster', 'pip', 'pc_num', 'lead_variant_id', 'num_vars']], on='var_cluster')

# pip weighted variance
e_nominal_pcsusie_subset['variance'] = e_nominal_pcsusie_subset['slope'].apply(np.square) * 100
e_nominal_pcsusie_subset['variance_weighted'] = e_nominal_pcsusie_subset['variance'] * e_nominal_pcsusie_subset['pip'] 

# group by phenotype (egene and cluster)
pcphenotype_evariance = e_nominal_pcsusie_subset.groupby('phenotype_id').agg({'variance_weighted':sum, 
                                                        'cluster_id':'first', 
                                                        'variant_id':list,
                                                        'egene_id':'first', 
                                                        'cs_id':'first', 
                                                        'lead_variant_id':'first',
                                                        'num_vars':'first'})

/local/scratch/klawren/slrmtmp.44431414/ipykernel_62380/1779498724.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  e_nominal_pcsusie_subset['var_egene_cluster'] = e_nominal_pcsusie_subset['variant_id'] + '_' + e_nominal_pcsusie_subset['phenotype_id']


KeyError: "['num_vars'] not in index"

In [9]:
pcphenotype_evariance

,variance_weighted,cluster_id,variant_id,egene_id,cs_id,lead_variant_id
phenotype_id,,,,,,
ENSG00000025708.13_ENSG00000177989.13_e_ENSG00000025708.13,0.246960,ENSG00000025708.13_ENSG00000177989.13,"[chr22_50504763_C_T_b38, chr22_50505337_G_A_b3...",ENSG00000025708.13,ENSG00000025708.13_ENSG00000177989.13_2,chr22_50519627_C_T_b38
ENSG00000025708.13_ENSG00000177989.13_e_ENSG00000177989.13,0.292988,ENSG00000025708.13_ENSG00000177989.13,"[chr22_50504763_C_T_b38, chr22_50505337_G_A_b3...",ENSG00000177989.13,ENSG00000025708.13_ENSG00000177989.13_2,chr22_50519627_C_T_b38
ENSG00000093000.18_ENSG00000100364.18_e_ENSG00000093000.18,0.350770,ENSG00000093000.18_ENSG00000100364.18,[chr22_45105543_G_A_b38],ENSG00000093000.18,ENSG00000093000.18_ENSG00000100364.18_1,chr22_45105543_G_A_b38
ENSG00000093000.18_ENSG00000100364.18_e_ENSG00000100364.18,0.114316,ENSG00000093000.18_ENSG00000100364.18,[chr22_45105543_G_A_b38],ENSG00000100364.18,ENSG00000093000.18_ENSG00000100364.18_1,chr22_45105543_G_A_b38
ENSG00000099904.15_ENSG00000234409.6_e_ENSG00000099904.15,0.183624,ENSG00000099904.15_ENSG00000234409.6,"[chr22_20153539_C_A_b38, chr22_20154469_C_G_b3...",ENSG00000099904.15,ENSG00000099904.15_ENSG00000234409.6_1,chr22_20154469_C_G_b38
ENSG00000099904.15_ENSG00000234409.6_e_ENSG00000234409.6,0.125595,ENSG00000099904.15_ENSG00000234409.6,"[chr22_20153539_C_A_b38, chr22_20154469_C_G_b3...",ENSG00000234409.6,ENSG00000099904.15_ENSG00000234409.6_1,chr22_20154469_C_G_b38
ENSG00000099974.7_ENSG00000099977.13_ENSG00000133433.10_ENSG00000240972.1_e_ENSG00000099974.7,0.205969,ENSG00000099974.7_ENSG00000099977.13_ENSG00000...,"[chr22_23924644_G_C_b38, chr22_23924680_G_A_b38]",ENSG00000099974.7,ENSG00000099974.7_ENSG00000099977.13_ENSG00000...,chr22_23924644_G_C_b38
ENSG00000099974.7_ENSG00000099977.13_ENSG00000133433.10_ENSG00000240972.1_e_ENSG00000099977.13,0.143449,ENSG00000099974.7_ENSG00000099977.13_ENSG00000...,"[chr22_23924644_G_C_b38, chr22_23924680_G_A_b38]",ENSG00000099977.13,ENSG00000099974.7_ENSG00000099977.13_ENSG00000...,chr22_23924644_G_C_b38
ENSG00000099974.7_ENSG00000099977.13_ENSG00000133433.10_ENSG00000240972.1_e_ENSG00000133433.10,0.316832,ENSG00000099974.7_ENSG00000099977.13_ENSG00000...,"[chr22_23924644_G_C_b38, chr22_23924680_G_A_b38]",ENSG00000133433.10,ENSG00000099974.7_ENSG00000099977.13_ENSG00000...,chr22_23924644_G_C_b38


In [24]:
e_nominal_pcsusie_subset[e_nominal_pcsusie_subset['phenotype_id']=='ENSG00000100033.16_ENSG00000183628.12_e_ENSG00000183628.12']


,phenotype_id,variant_id,start_distance,end_distance,af,ma_samples,ma_count,pval_nominal,slope,slope_se,cluster_id,var_cluster,egene_id,var_egene_cluster,cs_id,pip,pc_num,variance,variance_weighted
0,ENSG00000100033.16_ENSG00000183628.12_e_ENSG00...,chr22_18933556_A_G_b38,27528,-2854,0.111801,104,108,9.087594e-19,-0.047953,0.005163,ENSG00000100033.16_ENSG00000183628.12,chr22_18933556_A_G_b38_ENSG00000100033.16_ENSG...,ENSG00000183628.12,chr22_18933556_A_G_b38_ENSG00000100033.16_ENSG...,ENSG00000100033.16_ENSG00000183628.12_1,0.939348,1,0.229952,0.216004
2,ENSG00000100033.16_ENSG00000183628.12_e_ENSG00...,chr22_18940813_T_C_b38,34785,4403,0.110766,104,107,2.020764e-18,-0.048233,0.005252,ENSG00000100033.16_ENSG00000183628.12,chr22_18940813_T_C_b38_ENSG00000100033.16_ENSG...,ENSG00000183628.12,chr22_18940813_T_C_b38_ENSG00000100033.16_ENSG...,ENSG00000100033.16_ENSG00000183628.12_1,0.060652,1,0.232645,0.014110


In [23]:
e_nominal_pcsusie_subset['phenotype_id'].iloc[0]

'ENSG00000100033.16_ENSG00000183628.12_e_ENSG00000183628.12'

In [14]:
pcphenotype_evariance.groupby('cs_id').agg({'phenotype_id':list,
                                            'variance_weighted':list,
                                            'cluster_id':first,
                                            })

,variance_weighted,cluster_id,variant_id,egene_id,cs_id
phenotype_id,,,,,
ENSG00000025708.13_ENSG00000177989.13_e_ENSG00000025708.13,0.246960,ENSG00000025708.13_ENSG00000177989.13,chr22_50504763_C_T_b38,ENSG00000025708.13,ENSG00000025708.13_ENSG00000177989.13_2
ENSG00000025708.13_ENSG00000177989.13_e_ENSG00000177989.13,0.292988,ENSG00000025708.13_ENSG00000177989.13,chr22_50504763_C_T_b38,ENSG00000177989.13,ENSG00000025708.13_ENSG00000177989.13_2
ENSG00000093000.18_ENSG00000100364.18_e_ENSG00000093000.18,0.350770,ENSG00000093000.18_ENSG00000100364.18,chr22_45105543_G_A_b38,ENSG00000093000.18,ENSG00000093000.18_ENSG00000100364.18_1
ENSG00000093000.18_ENSG00000100364.18_e_ENSG00000100364.18,0.114316,ENSG00000093000.18_ENSG00000100364.18,chr22_45105543_G_A_b38,ENSG00000100364.18,ENSG00000093000.18_ENSG00000100364.18_1
ENSG00000099904.15_ENSG00000234409.6_e_ENSG00000099904.15,0.183624,ENSG00000099904.15_ENSG00000234409.6,chr22_20153539_C_A_b38,ENSG00000099904.15,ENSG00000099904.15_ENSG00000234409.6_1
ENSG00000099904.15_ENSG00000234409.6_e_ENSG00000234409.6,0.125595,ENSG00000099904.15_ENSG00000234409.6,chr22_20153539_C_A_b38,ENSG00000234409.6,ENSG00000099904.15_ENSG00000234409.6_1
ENSG00000099974.7_ENSG00000099977.13_ENSG00000133433.10_ENSG00000240972.1_e_ENSG00000099974.7,0.205969,ENSG00000099974.7_ENSG00000099977.13_ENSG00000...,chr22_23924644_G_C_b38,ENSG00000099974.7,ENSG00000099974.7_ENSG00000099977.13_ENSG00000...
ENSG00000099974.7_ENSG00000099977.13_ENSG00000133433.10_ENSG00000240972.1_e_ENSG00000099977.13,0.143449,ENSG00000099974.7_ENSG00000099977.13_ENSG00000...,chr22_23924644_G_C_b38,ENSG00000099977.13,ENSG00000099974.7_ENSG00000099977.13_ENSG00000...
ENSG00000099974.7_ENSG00000099977.13_ENSG00000133433.10_ENSG00000240972.1_e_ENSG00000133433.10,0.316832,ENSG00000099974.7_ENSG00000099977.13_ENSG00000...,chr22_23924644_G_C_b38,ENSG00000133433.10,ENSG00000099974.7_ENSG00000099977.13_ENSG00000...


now I do something similar for the pc_nominals, basically just adding to the susie df the pc var explained

In [16]:
# get a subset of variants that ended up in credible sets
pc_nominal_pcsusie_subset = pc_nominal[pc_nominal['var_cluster'].isin(pc_susie['var_cluster'])]

# I want to get a pip weighted variance for each egene
pc_nominal_pcsusie_subset['var_pcgene_cluster'] = pc_nominal_pcsusie_subset['variant_id'] + '_' + pc_nominal_pcsusie_subset['phenotype_id']

# merge in the data from the susie finemapping
pc_nominal_pcsusie_subset = pd.merge(pc_nominal_pcsusie_subset, pc_susie[['cs_id', 'var_cluster', 'pip', 'pc_num']], on='var_cluster')

# pip weighted variance
pc_nominal_pcsusie_subset['variance'] = pc_nominal_pcsusie_subset['slope'].apply(np.square) * 100
pc_nominal_pcsusie_subset['variance_weighted'] = pc_nominal_pcsusie_subset['variance'] * e_nominal_pcsusie_subset['pip'] 

# group by phenotype (egene and cluster)
pcphenotype_pcvariance = pc_nominal_pcsusie_subset.groupby('phenotype_id').agg({'variance_weighted':sum, 
                                                        'cluster_id':'first', 
                                                        'variant_id':'first',
                                                        'cs_id':'first'})

/local/scratch/klawren/slrmtmp.44431414/ipykernel_8853/2930575850.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pc_nominal_pcsusie_subset['var_pcgene_cluster'] = pc_nominal_pcsusie_subset['variant_id'] + '_' + pc_nominal_pcsusie_subset['phenotype_id']


In [20]:
pd.merge(pcphenotype_pcvariance, pcphenotype_evariance, on = 'cs_id')

,variance_weighted_x,cluster_id_x,variant_id_x,cs_id,variance_weighted_y,cluster_id_y,variant_id_y,egene_id
0,0.381875,ENSG00000025708.13_ENSG00000177989.13,chr22_50504763_C_T_b38,ENSG00000025708.13_ENSG00000177989.13_2,0.246960,ENSG00000025708.13_ENSG00000177989.13,chr22_50504763_C_T_b38,ENSG00000025708.13
1,0.381875,ENSG00000025708.13_ENSG00000177989.13,chr22_50504763_C_T_b38,ENSG00000025708.13_ENSG00000177989.13_2,0.292988,ENSG00000025708.13_ENSG00000177989.13,chr22_50504763_C_T_b38,ENSG00000177989.13
2,0.011984,ENSG00000025708.13_ENSG00000177989.13,chr22_50504763_C_T_b38,ENSG00000025708.13_ENSG00000177989.13_2,0.246960,ENSG00000025708.13_ENSG00000177989.13,chr22_50504763_C_T_b38,ENSG00000025708.13
3,0.011984,ENSG00000025708.13_ENSG00000177989.13,chr22_50504763_C_T_b38,ENSG00000025708.13_ENSG00000177989.13_2,0.292988,ENSG00000025708.13_ENSG00000177989.13,chr22_50504763_C_T_b38,ENSG00000177989.13
4,0.294142,ENSG00000093000.18_ENSG00000100364.18,chr22_45105543_G_A_b38,ENSG00000093000.18_ENSG00000100364.18_1,0.350770,ENSG00000093000.18_ENSG00000100364.18,chr22_45105543_G_A_b38,ENSG00000093000.18
...,...,...,...,...,...,...,...,...
63,0.019338,ENSG00000183066.14_ENSG00000198951.11,chr22_41992822_A_G_b38,ENSG00000183066.14_ENSG00000198951.11_1,0.252443,ENSG00000183066.14_ENSG00000198951.11,chr22_41992822_A_G_b38,ENSG00000198951.11
64,0.268754,ENSG00000243811.8_ENSG00000244509.3,chr22_39023260_C_T_b38,ENSG00000243811.8_ENSG00000244509.3_1,0.230780,ENSG00000243811.8_ENSG00000244509.3,chr22_39023260_C_T_b38,ENSG00000243811.8
65,0.268754,ENSG00000243811.8_ENSG00000244509.3,chr22_39023260_C_T_b38,ENSG00000243811.8_ENSG00000244509.3_1,0.124338,ENSG00000243811.8_ENSG00000244509.3,chr22_39023260_C_T_b38,ENSG00000244509.3
66,0.012858,ENSG00000243811.8_ENSG00000244509.3,chr22_39023260_C_T_b38,ENSG00000243811.8_ENSG00000244509.3_1,0.230780,ENSG00000243811.8_ENSG00000244509.3,chr22_39023260_C_T_b38,ENSG00000243811.8


I now have a table with a row for the % evar explained for each egene and one with the % pcvar for the pc-overall. I want to further annotate this with relevant info, like the pc-qtl lead variant to egene tss distance

In [11]:
# load in the gene information (start and strand are what I need)
full_gencode=pd.read_csv('/home/klawren/oak/pcqtls/data/references/processed_gencode.v26.GRCh38.genes.gtf', sep='\t', skiprows=range(6), 
            header=None, names=['chr', 'dataset', 'type', 'start','end', '.', 'strand', 'na', 'info'])

full_gencode = full_gencode[full_gencode['type']=='transcript']
full_gencode['transcript_id'] = full_gencode['info'].str.split(';').str[1].str.split('\"').str[-2]

# add in the start and end info
full_gencode['tss_start'] = np.where(full_gencode['strand'] == '+', full_gencode['start'], full_gencode['end'])
full_gencode['gene_end'] = np.where(full_gencode['strand'] == '-', full_gencode['start'], full_gencode['end'])

# filter to just the transcripts that are in the clusters
gene_ids = np.concatenate(overlap_df['cluster_id'].str.split('_'))
gid_gencode = full_gencode.set_index('transcript_id').loc[gene_ids]
gid_gencode = gid_gencode.drop_duplicates()

In [15]:
row = sum_variance_df.iloc[0]

In [19]:
egene_tss = gid_gencode.loc[row['egene_id']]['tss_start']
lead

KeyError: 'egene_id'